In [1]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.abs
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.{min, max, lit}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{ IntegerType, StructField, StructType, StringType, DoubleType}
val sqlContext = new SQLContext(sc)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1607009916543_0005,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.abs
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.{min, max, lit}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{IntegerType, StructField, StructType, StringType, DoubleType}
sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@53936c7f


In [2]:
val schema = new StructType().
add("artist_familiarity",DoubleType,true).
add("artist_hotttnesss",DoubleType,true).
add("artist_id",StringType,true).
add("artist_location",StringType,true).
add("artist_mbtags",StringType,true).
add("artist_mbtags_count",StringType,true).
add("artist_name",StringType,true).
add("artist_terms",StringType,true).
add("artist_terms_freq",StringType,true).
add("artist_terms_weight",StringType,true).
add("danceability",DoubleType,true).
add("duration",DoubleType,true).
add("end_of_fade_in",DoubleType,true).
add("energy",DoubleType,true).
add("key",DoubleType,true).
add("key_confidence",DoubleType,true).
add("loudness",DoubleType,true).
add("mode",DoubleType,true).
add("mode_confidence",DoubleType,true).
add("release",StringType,true).
add("segments_confidence",StringType,true).
add("segments_loudness_max",StringType,true).
add("segments_loudness_max_time",StringType,true).
add("segments_pitches",StringType,true).
add("segments_timbre",StringType,true).
add("similar_artists",StringType,true).
add("song_hotttnesss",DoubleType,true).
add("song_id",StringType,true).
add("start_of_fade_out",DoubleType,true).
add("tempo",DoubleType,true).
add("time_signature",IntegerType,true).
add("time_signature_confidence",DoubleType,true).
add("title",StringType,true).
add("track_id",StringType,true).
add("year",IntegerType,true)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema: org.apache.spark.sql.types.StructType = StructType(StructField(artist_familiarity,DoubleType,true), StructField(artist_hotttnesss,DoubleType,true), StructField(artist_id,StringType,true), StructField(artist_location,StringType,true), StructField(artist_mbtags,StringType,true), StructField(artist_mbtags_count,StringType,true), StructField(artist_name,StringType,true), StructField(artist_terms,StringType,true), StructField(artist_terms_freq,StringType,true), StructField(artist_terms_weight,StringType,true), StructField(danceability,DoubleType,true), StructField(duration,DoubleType,true), StructField(end_of_fade_in,DoubleType,true), StructField(energy,DoubleType,true), StructField(key,DoubleType,true), StructField(key_confidence,DoubleType,true), StructField(loudness,DoubleType,tru...

In [3]:
val msdf = sqlContext.read.format("com.databricks.spark.csv").
option("header", "false").
option("delimiter", "\t").
option("inferSchema", "false").
schema(schema).load("s3://millionsongprocessed/*.tsv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdf: org.apache.spark.sql.DataFrame = [artist_familiarity: double, artist_hotttnesss: double ... 33 more fields]


In [4]:
val usdf= sqlContext.read.format("com.databricks.spark.csv").
option("header","false").
option("delimiter", "\t").
load("s3://millionsongprocessed/train_triplets.txt")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

usdf: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 1 more field]


In [5]:
val list = List("user_id", "SongId", "PlayCount")
import spark.implicits._
val usdf2 = usdf.toDF(list:_*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list: List[String] = List(user_id, SongId, PlayCount)
import spark.implicits._
usdf2: org.apache.spark.sql.DataFrame = [user_id: string, SongId: string ... 1 more field]


In [6]:
// 'artist familiarity', 'artist hotttnesss', 'danceability',
//        'duration', 'end of fade in', 'energy', 'key', 'key confidence',
//        'loudness', 'mode', 'mode confidence', 'song hotttnesss',
//        'start of fade out', 'tempo', 'time signature',
//        'time signature confidence', 'year'

var msdf2 = msdf.withColumn("time_signature", when(col("time_signature_confidence")<0.5, 4).otherwise(col("time_signature")))
val msdf3 = msdf2.select($"artist_familiarity",$"duration",$"key", $"key_confidence", 
                         $"loudness",$"mode", $"tempo"/$"time_signature" as "speed", $"song_id")
var msdf4 = msdf3.na.drop()
val msdf6 = msdf4.withColumnRenamed("song_id","songID")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdf2: org.apache.spark.sql.DataFrame = [artist_familiarity: double, artist_hotttnesss: double ... 33 more fields]
msdf3: org.apache.spark.sql.DataFrame = [artist_familiarity: double, duration: double ... 6 more fields]
msdf4: org.apache.spark.sql.DataFrame = [artist_familiarity: double, duration: double ... 6 more fields]
msdf6: org.apache.spark.sql.DataFrame = [artist_familiarity: double, duration: double ... 6 more fields]


In [12]:
val topfeats = usdf2.join(msdf6, usdf2("SongId")===msdf6("songID")).orderBy(desc("PlayCount"))
topfeats.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

topfeats: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: string, SongId: string ... 9 more fields]
+--------------------+------------------+---------+-------------------+---------+----+--------------+--------+----+-----------------+------------------+
|             user_id|            SongId|PlayCount| artist_familiarity| duration| key|key_confidence|loudness|mode|            speed|            songID|
+--------------------+------------------+---------+-------------------+---------+----+--------------+--------+----+-----------------+------------------+
|92183d278210b8db1...|SOWEHOM12A6BD4E09E|       99| 0.5761477032059182|182.88281|10.0|         0.725| -12.812| 1.0|            36.83|SOWEHOM12A6BD4E09E|
|296634d2ac9b9524a...|SODDHOE12A58A7AC2F|       99| 0.6041098708079196|172.82567| 0.0|         0.565| -14.026| 0.0|         25.06525|SODDHOE12A58A7AC2F|
|73042a8a6394fd867...|SOWEHOM12A6BD4E09E|       99| 0.5761477032059182|182.88281|10.0|         0.725| -12.812| 1.0| 

In [13]:
val tf = topfeats.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tf: Array[org.apache.spark.sql.Row] = Array([792ee2e1cee612a77f0ae01b909cb5125fee2e06,SOHFGKG12A6701C429,99,0.7656664079340161,298.78812,7.0,0.743,-8.374,1.0,24.48325,SOHFGKG12A6701C429], [2d692f0c7eb6480bfb473227060b1294ff0a64f5,SOXAYEX12AF72A6225,99,0.989938569657492,247.09179,1.0,0.202,-8.802,0.0,40.599,SOXAYEX12AF72A6225], [1764daf9949ad3e9bc1b7db90a97336372289934,SOZIOED12AB018102A,99,0.6653200948181356,193.54077,8.0,0.21,-6.186,1.0,35.50275,SOZIOED12AB018102A])


In [17]:
msdf2.filter("artist_familiarity=0.718068955556776").select("title", "duration", "key_confidence").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+--------------+
|               title| duration|key_confidence|
+--------------------+---------+--------------+
|Find My Way Back ...|146.70322|         0.678|
|        Silent Night|137.97832|         0.524|
|           In A Tree|150.22975|          0.65|
|               Dream|211.64363|         0.578|
+--------------------+---------+--------------+

